In [11]:
# Generate yearly pnl time series given portfolio weights and historical timeseries of mutual funds
import os
import pandas as pd
import numpy as np

In [12]:
ret_ts_dir = r'../results/backtests/strategy_returns_timeseries'
rfr = 0.02

In [13]:
# Define performance metrics functions
def calculate_sharpe_ratio(returns, risk_free_rate=0.02):
    tret = calculate_annualized_return(returns)
    tvol = calculate_annualized_volatility(returns)
    excess_returns = tret - risk_free_rate
    return excess_returns / tvol

def calculate_sortino_ratio(returns, risk_free_rate=0.02):
    downside_std = np.std(returns[returns < 0])*np.sqrt(252)
    tret = calculate_annualized_return(returns)
    excess_returns = tret - risk_free_rate
    return excess_returns / downside_std if downside_std != 0 else np.nan

def calculate_max_drawdown(returns):
    cumulative = (1 + returns).cumprod()
    cumulative_max = cumulative.cummax()
    drawdown = (cumulative - cumulative_max) / cumulative_max
    return drawdown.min()

def calculate_annualized_return(returns):
    cumulative_return = (1 + returns).prod() - 1
    return cumulative_return

def calculate_annualized_volatility(returns):
    return np.std(returns) * np.sqrt(252)

In [14]:
# GOPALS Strategy 1 performance

# Initialize list to store metrics for each year
metrics_list = []
all_files = os.listdir(ret_ts_dir)

# filename = 'GOPALS_Strategy1_n_3_year_2020_pct_0.9.csv'
for filename in all_files:
    df = pd.read_csv(os.path.join(ret_ts_dir, filename), index_col='Date')
    df.columns = ['Ret']
    # Calculate metrics for the current year
    sharpe_ratio = calculate_sharpe_ratio(df['Ret'])
    sortino_ratio = calculate_sortino_ratio(df['Ret'])
    max_drawdown = calculate_max_drawdown(df['Ret'])
    annual_return = calculate_annualized_return(df['Ret'])
    annual_volatility = calculate_annualized_volatility(df['Ret'])
        
    # Append metrics to list
    metrics_list.append({'Strategy': filename,
            'Sharpe Ratio': sharpe_ratio,
            'Sortino Ratio': sortino_ratio,
            'Max Drawdown': max_drawdown,
            'Annual Return': annual_return,
            'Annual Volatility': annual_volatility})
# print(metrics_list)

In [15]:
pd.DataFrame(metrics_list).to_csv(r'../results/backtests/final_results.csv', index=False)